In [38]:
from collections import namedtuple

Entry = namedtuple('Entry', ['uniques', 'digits'])

with open('day_8_input') as f:
    entries = []
    for line in f.readlines():
        uniques_str, _, digits_str = line.strip().partition(' | ')
        uniques = uniques_str.split(' ')
        digits = digits_str.split(' ')
        entries.append(Entry(uniques, digits))

print(entries)

[Entry(uniques=['badf', 'gadfec', 'bgcad', 'ad', 'dbcfg', 'gcaeb', 'fecdgab', 'gad', 'bgcadf', 'efcdgb'], digits=['gcadb', 'ad', 'agd', 'deacfg']), Entry(uniques=['adgfeb', 'dagecf', 'ceadg', 'edabc', 'ceg', 'dfacebg', 'fgead', 'gfcd', 'becagf', 'gc'], digits=['acdeg', 'efbcag', 'agfcde', 'fagceb']), Entry(uniques=['fedgcb', 'gcadb', 'edgcf', 'af', 'fbcedga', 'gdafce', 'faebcg', 'acf', 'deaf', 'gdafc'], digits=['edafcg', 'fa', 'ecagfd', 'acdbg']), Entry(uniques=['gafced', 'gac', 'bdfeagc', 'dface', 'aedbg', 'agedc', 'cg', 'cgef', 'dafcgb', 'cbeafd'], digits=['dgcae', 'ceadf', 'cga', 'abdfce']), Entry(uniques=['bgda', 'cdg', 'dacbe', 'cfega', 'begcafd', 'febacd', 'degcba', 'adecg', 'edcfbg', 'gd'], digits=['dfecbg', 'dg', 'defgcb', 'gfcea']), Entry(uniques=['bc', 'gadebc', 'bdc', 'gebfd', 'cefb', 'bfgdcae', 'cgdfb', 'dcebgf', 'bgaedf', 'cfadg'], digits=['cdbega', 'begfda', 'gfcad', 'gbedf']), Entry(uniques=['gdcb', 'cgfeab', 'ecadfgb', 'gcbfe', 'dc', 'dcgef', 'becdfa', 'faged', 'ebdgcf'

In [39]:
interesting_lengths = set([2, 3, 4, 7])
num_interesting_digits = 0
for entry in entries:
    num_interesting_digits += len([d for d in entry.digits if len(d) in interesting_lengths])
print(num_interesting_digits)

245


In [47]:
digits_map = {
    'a': [0, 2, 3, 5, 6, 7, 8, 9],
    'b': [0, 4, 5, 6, 8, 9],
    'c': [0, 1, 2, 3, 4, 7, 8, 9],
    'd': [2, 3, 4, 5, 6, 8, 9],
    'e': [0, 2, 6, 8],
    'f': [0, 1, 3, 4, 5, 6, 7, 8, 9],
    'g': [0, 2, 3, 5, 6, 8, 9]
}

solver = {
    'abcefg': 0,
    'cf': 1,
    'acdeg': 2,
    'acdfg': 3,
    'bcdf': 4,
    'abdfg': 5,
    'abdefg': 6,
    'acf': 7,
    'abcdefg': 8,
    'abcdfg': 9
}

def get_counts(entry):
    counts = {}
    for letter in 'abcdefg':
        counts[letter] = 0
    for unique in entry.uniques:
        for letter in unique:
            counts[letter] = counts[letter] + 1
    return counts, {v: k for k, v in counts.items()}

def get_digit_by_size(entry, size):
    choices = [d for d in entry.uniques if len(d) == size]
    if (len(choices) != 1):
        raise Exception('Logic error')
    return choices[0]

def sub(val, minus):
    result = val
    for c in minus:
        result = result.replace(c, '')
    return result

def count_other(val, letters):
    for letter in letters:
        if letter not in val:
            return
    count = 0
    for c in val:
        if c not in letters:
            count += 1
    return count

def get_by_count_other(entry, letters, count):
    result = None
    for unique in entry.uniques:
        if count_other(unique, letters) == count:
            if result is not None:
                raise Exception('Logic error')
            result = unique
    if result is None:
        raise Exception('Logic error')
    return result

def extract(letter_map, letters):
    result = ''
    for letter in letters:
        result += letter_map[letter]
    return result
    
def solve_digit(inv_letter_map, digit):
    mapped = [inv_letter_map[d] for d in digit]
    normalized = ''.join(sorted(mapped))
    return solver[normalized]

def digits_to_num(digits):
    mult = 1
    result = 0
    for i in reversed(range(len(digits))):
        result += digits[i] * mult
        mult *= 10
    return result
    
full_sum = 0

for entry in entries:
    counts, counts_invert = get_counts(entry)

    # f is the only letter in 9 uniques
    letter_map = {}
    letter_map['f'] = counts_invert[9]

    # one is the only digit with 2 wires, since we know f we can find c
    one = get_digit_by_size(entry, 2)
    letter_map['c'] = sub(one, letter_map['f'])

    # seven is the only digit with 3 wires since we know cf we can find a
    seven = get_digit_by_size(entry, 3)
    letter_map['a'] = sub(seven, extract(letter_map, 'cf'))

    # four is the only digit with 4 wires since we know cf we can find
    # bd but not which is is which
    four = get_digit_by_size(entry, 4)
    bd = sub(four, extract(letter_map, 'cf'))

    abcdf = bd + extract(letter_map, 'acf')
    # nine will be the only unique that has abcdf and exactly 1 other wire (g)
    nine = get_by_count_other(entry, abcdf, 1)
    # subtract abcdf to get g
    letter_map['g'] = sub(nine, abcdf)

    # eight is the only digit with 7 wires
    eight = get_digit_by_size(entry, 7)

    # subtract abcdfg to find e
    abcdfg = abcdf + letter_map['g']
    letter_map['e'] = sub(eight, abcdfg)

    acefg = extract(letter_map, 'acefg')

    # zero is the only unique that has acefg and exactly 1 other wire (b)
    zero = get_by_count_other(entry, acefg, 1)
    letter_map['b'] = sub(zero, acefg)

    # now, since we know b, we can take apart bd to find d
    letter_map['d'] = sub(bd, letter_map['b'])

    inv_letter_map = {v: k for k, v in letter_map.items()}
    digits = [solve_digit(inv_letter_map, digit) for digit in entry.digits]
    print(digits)
    full_sum += digits_to_num(digits)
print(full_sum)

[3, 1, 7, 0]
[3, 0, 9, 0]
[9, 1, 9, 2]
[3, 5, 7, 6]
[0, 1, 0, 2]
[0, 6, 2, 5]
[6, 7, 6, 5]
[7, 1, 1, 6]
[3, 9, 5, 4]
[2, 2, 6, 8]
[5, 5, 8, 7]
[9, 3, 6, 9]
[4, 8, 4, 8]
[3, 8, 2, 8]
[3, 9, 6, 5]
[8, 1, 5, 9]
[9, 4, 8, 8]
[8, 2, 6, 7]
[2, 3, 0, 3]
[5, 4, 1, 3]
[9, 1, 3, 0]
[1, 1, 5, 6]
[5, 7, 9, 9]
[2, 7, 2, 3]
[7, 1, 3, 3]
[3, 9, 4, 8]
[2, 8, 9, 4]
[3, 8, 2, 5]
[5, 2, 2, 5]
[9, 3, 0, 5]
[0, 9, 9, 1]
[3, 2, 2, 0]
[5, 5, 9, 3]
[1, 9, 3, 6]
[7, 2, 4, 0]
[7, 4, 1, 6]
[9, 8, 2, 2]
[9, 0, 5, 6]
[0, 4, 6, 5]
[0, 1, 1, 9]
[1, 9, 1, 5]
[5, 9, 5, 8]
[9, 0, 4, 5]
[5, 0, 3, 0]
[4, 5, 4, 9]
[6, 0, 9, 9]
[6, 2, 6, 8]
[0, 3, 0, 3]
[9, 0, 5, 7]
[8, 2, 7, 1]
[0, 8, 9, 7]
[2, 1, 0, 6]
[8, 6, 7, 5]
[9, 5, 3, 6]
[6, 7, 7, 0]
[0, 3, 7, 9]
[4, 5, 8, 3]
[6, 1, 9, 9]
[9, 3, 0, 5]
[5, 0, 4, 3]
[5, 5, 0, 4]
[5, 2, 2, 5]
[1, 2, 9, 1]
[9, 9, 4, 2]
[2, 3, 6, 3]
[2, 8, 9, 2]
[6, 9, 9, 8]
[9, 4, 6, 3]
[5, 5, 0, 0]
[1, 5, 2, 7]
[7, 2, 9, 9]
[9, 9, 5, 5]
[5, 1, 5, 2]
[3, 5, 3, 6]
[8, 9, 2, 0]
[9, 4, 8, 2]
[3, 3, 8, 4]